In [1]:
# to make the export go to the fitbit app, click on your icon, and order a google takeout 
# took like 8 hours the last time

In [1]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeWorkingHRDfParquet

import pandas as pd

fitbitHRWorkingDataPath = workingDataPath + 'fitbit/hr/'

# Get the list of all files and directories
exportFilePath = exportsDataPath + "fitbit/17-9-24/takeout-20240917T195619Z-001/Takeout/Fitbit/Global Export Data/"
dir_list = os.listdir(exportFilePath)


/home/chowder/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# read in the existing files and make the existing df
workingDataFiles = os.listdir(fitbitHRWorkingDataPath)
columnNames = ["sampleDT", "confidence", "value"]
dfSoFar = pd.DataFrame(columns=columnNames).set_index("sampleDT")

In [3]:
for dataFileName in workingDataFiles:
    dfSoFar = pd.concat([dfSoFar, pd.read_parquet(fitbitHRWorkingDataPath + dataFileName)]) 

dfSoFar = dfSoFar[~dfSoFar.index.duplicated(keep="first")].sort_index()

print(f"read in {len(dfSoFar)} rows from {len(workingDataFiles)} files")

read in 7000000 rows from 38 files


In [4]:
# get a list of unique dates in the index
#   removing the latest 3 days since they might be incomplete
if len(dfSoFar) > 0:
    datesSoFar = sorted(list(set(dfSoFar.index.date)))[:-3]
    print(datesSoFar[-1])
    hrFilenames = [x for x in dir_list if x.split("-")[0] == "heart_rate"]
    hrFilenames = [x for x in hrFilenames if pd.to_datetime(x[11:-5]).date() not in datesSoFar]
else:
    hrFilenames = [x for x in dir_list if x.split("-")[0] == "heart_rate"]

2023-07-29


In [5]:
#takes like 50 seconds to make 16 million rows
import json
import pytz
from datetime import datetime

def to_iso(s):
    return '20' + s[6:8] + '-' + s[0:2] + '-' + s[3:5] + s[8:]

samplesCount = 0
samplesList = []
for fn in hrFilenames:
    data = json.load(open(exportFilePath + fn))
    for row in data:
        sampleDT = datetime.fromisoformat(to_iso(row["dateTime"]))
        samplesList.append([sampleDT, 
                            row["value"]["confidence"], 
                            row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 1_000_000 == 0:
            print(f"added {samplesCount} samples so far")
print(f"added {samplesCount} samples")

added 1000000 samples so far
added 2000000 samples so far
added 3000000 samples so far
added 4000000 samples so far
added 5000000 samples so far
added 6000000 samples so far
added 7000000 samples so far
added 8000000 samples so far
added 9000000 samples so far
added 9084269 samples


In [6]:
import pandas as pd

columnNames = ["sampleDT", "confidence", "value"]
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf.index = fitbitHRdf.index.tz_localize('UTC')

fitbitHRdf.index = fitbitHRdf.index.tz_convert("US/Arizona")

fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

confidence    uint8
value         uint8
dtype: object

In [7]:
# concatenate the df's
# remove duplicate indexes
fitbitHRdf = pd.concat([dfSoFar, fitbitHRdf])
fitbitHRdf = fitbitHRdf[~fitbitHRdf.index.duplicated(keep="first")].sort_index()

In [ ]:
#new fileNames = <start timestamp>_<end timestamp>_<8 hex hash>_.parquet.gzip

In [ ]:
import hashlib
import pickle

# Function to compute a short hash of a Python object
def short_hash(obj, length=8):
    # Serialize the object using pickle
    obj_bytes = pickle.dumps(obj)
    
    # Compute MD5 hash of the serialized object
    hash_obj = hashlib.md5(obj_bytes)
    
    # Return the hash truncated to the specified length
    return hash_obj.hexdigest()[:length]

In [45]:
def writeToFileName(fileName, HRDf, interval, minRowsPerFile):
    if(short_hash(HRDf[starttime:endtime].values) != fileName.split('_')[2]):
        #combine, sort and deduplicate
        dftoWrite = pd.concat([HRDf[starttime:endtime], pd.read_parquet(fitbitHRWorkingDataPath + dataFileName)]) 
        dftoWrite = dftoWrite[~dftoWrite.index.duplicated(keep="first")].sort_index()

        #combine the



In [ ]:
#segmenting a big portion
# cut off half max length segments untill the last one

import math
def writeWorkingHRDfParquet(deviceName, HRDf, clearFiles = True):
    # first check if there are any files already there
    fileNames = sorted(os.listdir(workingDataHRPath))
    # what are those file names and what ranges do they cover
    firstFilesEndtime = pd.to_datetime(fileNames[0].split('_')[1])
    intersectionDF = HRDf[HRDf.index[0]:filesStarttime]

    chunkToWrite(HRDf)


    filesToadd = []
    #add all the rows it can to the existing file range
    for fNumber, fName in enumerate(fileNames):
        startTime = pd.to_datetime(fName.split('_')[0])
        endTime = pd.to_datetime(fName.split('_')[1])
        intersectionDF = HRDf[startTime:endTime]
        if len(intersectionDF) > 0:
            pass
    
    # fill in the gaps by incorporating them into the previous file, exept for the first one


    # calculate the dataframe to save based on that interval and calc hash
    # read the interval file and calc hash
    # if the hashes are the same then remove that interval from the HRDf
        # but also make sure we don't make a file that encompasses the other one
    
    # we can go in chronological order for the files
    # for any files bedfore the start of the Df, pop those files off
    # for any samples before the existing files, merge them into the first file and segment
    # for each file, check that there are no gaps from the last file

    # for each file merge prevous data since last file and split, merge last split into a bigger file

    pass

In [ ]:
getRowsPerFile(fileName = None, targetFileSize = 2 * 1024 * 1024)

In [ ]:
def writeWorkingHRDfParquet(deviceName, HRDf, clearFiles = True):
    # let's just start with if files
    fileNames = sorted(os.listdir(workingDataHRPath))

    #get the size and rows of the first file to calculate the 

    for fileNum, fileName in fileNames:
        if fileNum == 0:
            startTime = HRDf.index[0]
        else:
            startTime = pd.to_datetime(fileName.split('_')[0])
        
        if len(fileNames) == 1 or fileNum == len(fileNames) - 1:
            endTime = HRDf.index[-1]
        else:
            endTime = pd.to_datetime(fileNames[fileNum + 1].split('_')[0])

        if shortHash(HRDf[startTime:endTime].values) != fileName.split('_')[2]:
            saveRows(HRDf[startTime:endTime])


                
            
        
        
                



In [8]:
writeWorkingHRDfParquet('fitbit', fitbitHRdf.iloc[5000000:12000000])

a million rows took up 2.7354907989501953 MB
putting 365565 rows per file
2021-08-12T042809-0700_2021-08-12T042809-0700.parquet.gzip
0
the start row and parsed time is 0 2021-08-12 04:28:09-07:00
the time of the start row is 2021-08-12 08:41:48-07:00
the end row and  parsed  time is 0 2021-08-12 04:28:09-07:00
the time of the end row is 2021-08-12 08:41:48-07:00
saving rows 0 to 0
to a file named 2021-08-12T084148-0700_2021-08-12T084148-0700.parquet.gzip
2021-08-12T042809-0700_2021-08-22T211118-0700.parquet.gzip
1
the start row and parsed time is 0 2021-08-12 04:28:09-07:00
the time of the start row is 2021-08-12 08:41:48-07:00
the end row and  parsed  time is 114139 2021-08-22 21:11:18-07:00
the time of the end row is 2021-08-22 21:11:23-07:00
saving rows 0 to 114139
to a file named 2021-08-12T084148-0700_2021-08-22T211123-0700.parquet.gzip
2021-08-12T042820-0700_2021-09-14T203957-0700.parquet.gzip
2
the start row and parsed time is 0 2021-08-12 04:28:20-07:00
the time of the start ro

In [9]:
writeWorkingHRDfParquet('fitbit', fitbitHRdf)

a million rows took up 2.7364563941955566 MB
putting 365436 rows per file
2022-01-04T094802-0700_2022-02-13T113319-0700.parquet.gzip
0
the start row and parsed time is 0 2022-01-04 09:48:02-07:00
the time of the start row is 2020-05-13 10:02:19-07:00
the end row and  parsed  time is 6827675 2022-02-13 11:33:19-07:00
the time of the end row is 2022-02-13 11:33:24-07:00
6827675 is too many rows writing 0 to 365436
saving rows 0 to 365436
to a file named 2020-05-13T100219-0700_2020-06-15T175149-0700.parquet.gzip
6462239 is too many rows writing 365436 to 730872
saving rows 365436 to 730872
to a file named 2020-06-15T175149-0700_2020-07-19T103931-0700.parquet.gzip
6096803 is too many rows writing 730872 to 1096308
saving rows 730872 to 1096308
to a file named 2020-07-19T103931-0700_2020-08-22T011641-0700.parquet.gzip
5731367 is too many rows writing 1096308 to 1461744
saving rows 1096308 to 1461744
to a file named 2020-08-22T011641-0700_2020-09-25T101037-0700.parquet.gzip
5365931 is too ma

In [11]:
"2023-06-27T010355-0700_2023-07-10T061055-0700.parquet.gzip".replace('.', '_').split('_')

['2023-06-27T010355-0700', '2023-07-10T061055-0700', 'parquet', 'gzip']

In [9]:
import hashlib
import pickle

# Function to compute a short hash of a Python object
def short_hash(obj, length=8):
    # Serialize the object using pickle
    obj_bytes = pickle.dumps(obj)
    
    # Compute MD5 hash of the serialized object
    hash_obj = hashlib.md5(obj_bytes)
    
    # Return the hash truncated to the specified length
    return hash_obj.hexdigest()[:length]

# Example object

# Compute short hash
hash_value = short_hash(fitbitHRdf)

print(f"Short hash: {hash_value}")

Short hash: 34df307f


In [25]:
hash_value = short_hash(fitbitHRdf)

print(f"Short hash: {hash_value}")

Short hash: 6f7d2095


In [29]:
hash_value = short_hash(fitbitHRdf)

print(f"Short hash: {hash_value}")

Short hash: b07419fc


In [32]:
hash_value = short_hash(fitbitHRdf)

print(f"Short hash: {hash_value}")

Short hash: 6f7d2095


In [43]:
hash_value = short_hash(fitbitHRdf.values)

print(f"Short hash: {hash_value}")

Short hash: 60ece183


In [44]:
dfCopy = fitbitHRdf.copy()
hash_value = short_hash(dfCopy.values)

print(f"Short hash: {hash_value}")

Short hash: 60ece183


In [14]:
fitbitHRdf.iloc[5000000]['value']

61

In [19]:
tatgetIndex = fitbitHRdf.index[5000000]
tatgetIndex

Timestamp('2021-08-12 08:41:48-0700', tz='US/Arizona')

In [20]:
fitbitHRdf.loc[tatgetIndex]['value']

61

In [21]:
fitbitHRdf.loc[tatgetIndex]['value'] = 60

/tmp/ipykernel_2866694/2761556078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fitbitHRdf.loc[tatgetIndex]['value'] = 60


In [33]:
fitbitHRdf.at[tatgetIndex, 'value'] = 61

In [34]:
fitbitHRdf.loc[tatgetIndex]['value']

61